In [1]:
import requests
import json
import pandas as pd
import numpy as np
import pymysql
import os

In [8]:
# 1. 영화 id 정보추출을 위해 영화정보 파일 가져오기 

tmdb_movie_list = pd.read_csv('../data-files/total_tmdbmovielist.csv',index_col=0) 

C:\Users\bonomania\.conda\envs\ykseo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
tmdb_movie_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630186 entries, 0 to 630185
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   adult                  630158 non-null  object 
 1   backdrop_path          150102 non-null  object 
 2   belongs_to_collection  11995 non-null   object 
 3   budget                 630157 non-null  object 
 4   genres                 630155 non-null  object 
 5   homepage               69801 non-null   object 
 6   id                     630156 non-null  object 
 7   imdb_id                396414 non-null  object 
 8   original_language      630157 non-null  object 
 9   original_title         630130 non-null  object 
 10  overview               25238 non-null   object 
 11  popularity             630119 non-null  object 
 12  poster_path            426255 non-null  object 
 13  production_companies   630119 non-null  object 
 14  production_countries   630088 non-nu

In [10]:
tmdb_movie_list.shape

(630186, 25)

In [11]:
tmdb_movie_list.head(3)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,NaN,0.0,"[{'id': 18, 'name': '드라마'}, {'id': 80, 'name':...",NaN,2.0,tt0094675,fi,Ariel,...,1988-10-21,0.0,73.0,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,NaN,아리엘,False,6.9,150.0
1,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,NaN,0.0,"[{'id': 18, 'name': '드라마'}, {'id': 35, 'name':...",NaN,3.0,tt0092149,fi,Varjoja paratiisissa,...,1986-10-17,0.0,76.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,천국의 그림자,False,7.2,149.0
2,False,/bCXLsO2MJWzYYBJoavGOZN5iqGn.jpg,NaN,4000000.0,"[{'id': 80, 'name': '범죄'}, {'id': 35, 'name': ...",https://www.miramax.com/movie/four-rooms/,5.0,tt0113101,en,Four Rooms,...,1995-12-09,4257354.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,네 가지 이야기들로 이루어진 옴니버스 영화,포룸,False,5.7,2035.0


In [12]:
# 1. 영화테이블에서 장르id만 추출하여 genre_ids칼럼을 만들기  

error_list_x = []

def genres_str_to_list(genres_str):
    import json
    # print(genres_str)
    genre_id_list = []
    try :
        genres_list = json.loads(genres_str.replace("\'", "\""))
        for genre in genres_list:
            genre_id_list.append(genre['id'])

    except:
        error_list_x.append(genres_str)
    
    return genre_id_list

tmdb_movie_list['genre_ids'] = tmdb_movie_list["genres"].map(genres_str_to_list)

tmdb_movie_list.tail(5)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genre_ids
630181,False,NaN,NaN,2000.0,"[{'id': 27, 'name': '공포'}]",https://www.youtube.com/watch?v=ARtMOUG31jQ&li...,879995.0,NaN,en,Lore School - Episode 1: The Shrieker,...,0.0,9.0,[],Released,NaN,Lore School - Episode 1: The Shrieker,False,0.0,0.0,[27]
630182,True,NaN,NaN,0.0,[],NaN,879996.0,tt0296694,en,Джоанни Пневматик,...,0.0,79.0,[],Released,NaN,Джоанни Пневматик,False,0.0,0.0,[]
630183,False,/4R9Q9xxDq68hxSPzIR2C9R75Q6a.jpg,NaN,0.0,"[{'id': 18, 'name': '드라마'}, {'id': 35, 'name':...",NaN,879997.0,tt0060871,sv,Pälsen,...,0.0,25.0,"[{'english_name': 'No Language', 'iso_639_1': ...",Released,NaN,Pälsen,False,0.0,0.0,"[18, 35, 10770]"
630184,True,NaN,NaN,0.0,[],NaN,879998.0,NaN,en,Venial Temptation,...,0.0,109.0,[],Released,NaN,Venial Temptation,False,0.0,0.0,[]
630185,False,NaN,NaN,0.0,[],https://www.alqabas.com/,879999.0,NaN,ar,بلاغ الكويت ما بعد العاصفه,...,0.0,52.0,"[{'english_name': 'Arabic', 'iso_639_1': 'ar',...",Released,NaN,بلاغ الكويت ما بعد العاصفه,False,0.0,0.0,[]


In [14]:
print(len(error_list_x))

62


In [16]:
# genre_ids가 없는 행 제거 

print( type(tmdb_movie_list['genre_ids'][0]))
tmdb_movie_list['genre_ids'].map(lambda x : True if len(x) > 0 else False)
tmdb_movie_list_1=tmdb_movie_list[tmdb_movie_list['genre_ids'].map(lambda x : True if len(x) > 0 else False)]
tmdb_movie_list_1


<class 'list'>


,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genre_ids
0,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,NaN,0.0,"[{'id': 18, 'name': '드라마'}, {'id': 80, 'name':...",NaN,2.0,tt0094675,fi,Ariel,...,0.0,73.0,"[{'english_name': 'German', 'iso_639_1': 'de',...",Released,NaN,아리엘,False,6.9,150.0,"[18, 80, 35]"
1,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,NaN,0.0,"[{'id': 18, 'name': '드라마'}, {'id': 35, 'name':...",NaN,3.0,tt0092149,fi,Varjoja paratiisissa,...,0.0,76.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,천국의 그림자,False,7.2,149.0,"[18, 35]"
2,False,/bCXLsO2MJWzYYBJoavGOZN5iqGn.jpg,NaN,4000000.0,"[{'id': 80, 'name': '범죄'}, {'id': 35, 'name': ...",https://www.miramax.com/movie/four-rooms/,5.0,tt0113101,en,Four Rooms,...,4257354.0,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,네 가지 이야기들로 이루어진 옴니버스 영화,포룸,False,5.7,2035.0,"[80, 35]"
3,False,/5aXp2s4l6g5PcMMesIj63mx8hmJ.jpg,NaN,21000000.0,"[{'id': 28, 'name': '액션'}, {'id': 53, 'name': ...",NaN,6.0,tt0107286,en,Judgment Night,...,12136938.0,110.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,킬러 나이트,False,6.5,215.0,"[28, 53, 80]"
4,False,NaN,NaN,42000.0,"[{'id': 99, 'name': '다큐멘터리'}]",http://www.lifeinloops.com,8.0,tt0825671,en,Life in Loops (A Megacities RMX),...,0.0,80.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Life in Loops (A Megacities RMX),False,7.5,18.0,[99]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630176,False,NaN,NaN,0.0,"[{'id': 53, 'name': '스릴러'}]",NaN,879990.0,tt11818818,en,God's Country,...,0.0,0.0,[],Post Production,NaN,God's Country,False,0.0,0.0,[53]
630179,False,NaN,NaN,0.0,"[{'id': 80, 'name': '범죄'}, {'id': 18, 'name': ...",NaN,879993.0,tt0348047,en,The Skull,...,0.0,0.0,"[{'english_name': 'No Language', 'iso_639_1': ...",Released,NaN,The Skull,False,0.0,0.0,"[80, 18, 27, 53]"
630180,False,/yPfhwt7bRte0sHmXgDTGiedmFyu.jpg,NaN,0.0,"[{'id': 16, 'name': '애니메이션'}, {'id': 12, 'name...",NaN,879994.0,NaN,zh,魔法奇程,...,0.0,0.0,"[{'english_name': 'Mandarin', 'iso_639_1': 'zh...",Released,NaN,魔法奇程,False,0.0,0.0,"[16, 12]"
630181,False,NaN,NaN,2000.0,"[{'id': 27, 'name': '공포'}]",https://www.youtube.com/watch?v=ARtMOUG31jQ&li...,879995.0,NaN,en,Lore School - Episode 1: The Shrieker,...,0.0,9.0,[],Released,NaN,Lore School - Episode 1: The Shrieker,False,0.0,0.0,[27]


In [17]:
tmdb_movie_list_1.shape

(384533, 26)

In [18]:
movie_genre_ids_df=tmdb_movie_list_1[['id','genre_ids']]
movie_genre_ids_df.head()

,id,genre_ids
0,2.0,"[18, 80, 35]"
1,3.0,"[18, 35]"
2,5.0,"[80, 35]"
3,6.0,"[28, 53, 80]"
4,8.0,[99]


In [ ]:
##영화_장르 연결테이블 2)movie_id, genre_id삽입

movie_genre_ids_df=tmdb_movie_list_1[['id','genre_ids']]

import pymysql

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")
cursor = conn.cursor()
# cursor.execute('DELETE FROM movie_genre')

error_list2 = []
for movie_genre_ids in movie_genre_ids_df.values:
    #print(movie_genre_ids[1])
    #break
    current_genre_id = None
    try:
        movie_id = movie_genre_ids[0]
        for genre_id in movie_genre_ids[1]:
            current_genre_id = genre_id
            sql= "INSERT INTO movie_genre VALUES (%s, %s)"
            cursor.execute(sql, (movie_id, genre_id))
            print((movie_id, genre_id))
    except:
        error_list2.append((movie_id, current_genre_id))
conn.commit()
cursor.close()

conn.close()

In [ ]:
print(len(error_list2))
error_list2